In [1]:
# reload code if library changes
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import numpy as np

In [3]:
import datajoint as dj
from workflow_calcium_imaging.pipeline import *
populate_settings = {'display_progress': True}

[2023-02-28 11:27:03,415][INFO]: Connecting phan@127.0.0.1:3306
[2023-02-28 11:27:03,507][INFO]: Connected phan@127.0.0.1:3306


In [4]:
# move to library
def get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init):
    all_subject_str = []
    all_session_str = [] # this will be list of lists - each nested list corresponding to one subject
    all_datetime_str = [] # for now hardcoded! (figure out how to do programmatically)

    count = 0
    for subject_str in os.listdir(root_data_dir):
        if os.path.isdir(f'{root_data_dir}/{subject_str}'):

            print(f'Subject: {subject_str}')
            all_subject_str.append(subject_str)

            all_subject_session_str = [] # sessions for this particular subject
            all_subject_datetime_str = []
            for subject_session_str in os.listdir(root_data_dir + '/' + subject_str):
                all_subject_session_str.append(subject_session_str)
                fake_session_datetime_str = fake_session_datetime_str_init[:18] + str(count) + '.000' # making fake unique time
                print('\n\n\nIMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.\n\n\n')
                print(fake_session_datetime_str)
                all_subject_datetime_str.append(fake_session_datetime_str) # here it is fake
                count += 1

            print(f'Identified sessions for subject {subject_str}: {all_subject_session_str}')

            all_session_str.append(all_subject_session_str) 
            all_datetime_str.append(all_subject_datetime_str) 
        
    return all_subject_str, all_session_str, all_datetime_str

### Clear previous s2p entries

In [5]:
imaging.ProcessingParamSet.delete()
imaging.Curation.delete()

[2023-02-28 11:27:03,723][INFO]: Deleting 0 rows from `phan_imaging`.`#processing_param_set`
[2023-02-28 11:27:03,733][INFO]: Deleting 0 rows from `phan_imaging`.`curation`


Nothing to delete.
Nothing to delete.


0

In [6]:
# custom function to populate database
root_data_dir = dj.config['custom']['imaging_root_data_dir']
fake_session_datetime_str_init = '2002-01-01 12:00:00.000' # making up session datetime (to query data use either the session/folder name or PraireView metadata)
all_subject_str, all_session_str, all_datetime_str = get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init)


Subject: jm003



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:00.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:01.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:02.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the 

Automated Suite2p

## Note from Jure to Phan: everything below here is super preliminary, i'm debugging some issues with suite2p still (the bottom cell where params_suite2p are defined will change a lot, and also later on i am only testing all of this on a single dataset specified manually (see the section titled 'Running suite2p for one animal'))

In [7]:
choose_params = 'ops_jm_30hz_1plane' # number of channels, imaging rate and number of planes is overwritten by datajoint (see code for imaging.Processing.populate)

params_suite2p = np.load(f'/home/cossart/deve-networks/scripts/s2p_ops/{choose_params}.npy', allow_pickle=True).item()

params_suite2p['look_one_level_down'] = 0
params_suite2p['input_format'] = 'bruker'
params_suite2p['bruker'] = True
params_suite2p['force_sktiff'] = True
params_suite2p['two_step_registration'] = 0.0
params_suite2p['fast_disk'] = ['/suite2p_fast_disk']
params_suite2p

{'suite2p_version': '0.10.3',
 'look_one_level_down': 0,
 'fast_disk': ['/suite2p_fast_disk'],
 'delete_bin': True,
 'mesoscan': False,
 'bruker': True,
 'bruker_bidirectional': False,
 'h5py': [],
 'h5py_key': 'data',
 'nwb_file': '',
 'nwb_driver': '',
 'nwb_series': '',
 'save_path0': [],
 'save_folder': [],
 'subfolders': [],
 'move_bin': False,
 'nplanes': 1,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 0.1,
 'fs': 30.0,
 'force_sktiff': True,
 'frames_include': -1,
 'multiplane_parallel': 0.0,
 'ignore_flyback': [],
 'preclassify': 0.0,
 'save_mat': False,
 'save_NWB': 0.0,
 'combined': 1.0,
 'aspect': 1.0,
 'do_bidiphase': False,
 'bidiphase': 0.0,
 'bidi_corrected': False,
 'do_registration': 1,
 'two_step_registration': 0.0,
 'keep_movie_raw': True,
 'nimg_init': 300,
 'batch_size': 500,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': True,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 0.0,
 'smooth_sigma': 1.15,
 'th_badframes': 1.0,
 'norm_frames':

In [8]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    paramset_idx=0, 
    params=params_suite2p,
    paramset_desc=f'Calcium imaging analysis with Suite2p using {choose_params} parameters')

In [9]:
from workflow_calcium_imaging import process
process.run()


---- Populate imported and computed tables ----


ScanInfo:   0%|          | 0/12 [00:00<?, ?it/s]




Scan_filepaths:
/media/cossart/DATA/dj_cossart/data/jm010/2022-06-15_c/TSeries-05182022-1006-003/TSeries-05182022-1006-003_Cycle00001_Ch1_000001.ome.tif






IMPORTANT: JM: I also changed default number of roi from 1 to 0, because 1 throws an error with auto-populating suite2p





ScanInfo:   0%|          | 0/12 [00:02<?, ?it/s]

z_fields.size: 1
n_depths: 2
z_step: 200.0



IMPORTANT: here I (JM) hardcoded the subindex because of the issue of two z devices (see: https://github.com/datajoint/element-interface/issues/77)






IMPORTANT: another issue can occur here, sometimes Prairie view writes the z position of planes wrongly (even though the z changes, it logs the same depth for the two planes)



Error occured in asserting "z_fields.size == n_depths", moving to next dataset (this is probably because of a Prairie view bug, sometimes it writes the z position of planes wrongly (even though the z changes, it logs the same depth for the two planes))
Overwriting n_depths to be equal to z_fields.size...
{'num_fields': 1, 'num_channels': 2, 'num_planes': 1, 'num_frames': 24000, 'num_rois': 0, 'x_pos': None, 'y_pos': None, 'z_pos': None, 'frame_rate': 30.22341267543331, 'bidirectional': False, 'bidirectional_z': True, 'scan_datetime': datetime.datetime(2022, 6, 15, 18, 56, 46), 'usecs_per_line': 63.143, 'scan_durati

DataError: (1265, "Data truncated for column 'field_z' at row 1")

## Running suite2p for one animal

In [ ]:
session.SessionDirectory()

In [ ]:
session_key = (session.SessionDirectory & 'subject="jm007"').fetch('KEY')[0]
print(session_key )
session_dir_key = (session.SessionDirectory & 'subject="jm007"').fetch('session_dir')[0]

imaging.ProcessingTask.insert1(dict(session_key, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    processing_output_dir=f'{session_dir_key}', 
                                    task_mode='trigger'))

## Populate `imaging.Processing`

In [ ]:
imaging.Processing.populate(**populate_settings);

In [ ]:
imaging.Processing()

## Insert new Curation following the ProcessingTask

+ The next step in the pipeline is the curation of motion corection and segmentation results.

+ If a manual curation was implemented, an entry needs to be manually inserted into the table `imaging.Curation`, which specifies the directory to the curated results in `curation_output_dir`. 

+ If we would like to use the processed outcome directly, an entry is also needed in `imaging.Curation`. A method `create1_from_processing_task` was provided to help this insertion. It copies the `processing_output_dir` in `imaging.ProcessingTask` to the field `curation_output_dir` in the table `imaging.Curation` with a new `curation_id`.

    + In this example, we create/insert one `imaging.Curation` for each `imaging.ProcessingTask`, specifying the same output directory.

    + To this end, we could also make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [ ]:
imaging.Curation.heading

In [ ]:
# temp just to try the 2 plane 2 channel:
import datetime
foo_dict = {'subject': 'jm007', 'session_datetime': datetime.datetime(2002, 1, 1, 12, 0, 8)}

imaging.Curation.insert1(dict(subject=foo_dict['subject'], 
                                session_datetime=foo_dict['session_datetime'], 
                                scan_id=0,
                                paramset_idx=0,
                                curation_id=0,
                                curation_time=foo_dict['session_datetime'], 
                                curation_output_dir='jm007/2022-06-14_a/TSeries-05182022-1006-001/suite2p',
                                manual_curation=False,
                                curation_note=''))

In [ ]:
for (i, subject_str) in enumerate(all_subject_str):
    #for (j, subject_session_str) in enumerate(all_session_str[i]): # ONLY RUNNING FOR FIRST SESSION
    
    j = 0
    subject_session_str = all_session_str[i][j]
    
    session_datetime_str = all_datetime_str[i][j]

    imaging.Curation.insert1(dict(subject=subject_str, 
                                    session_datetime=session_datetime_str, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    curation_id=0,
                                    curation_time=session_datetime_str, 
                                    curation_output_dir=f'{subject_str}/{subject_session_str}/TSeries/suite2p',
                                    manual_curation=False,
                                    curation_note=''))


    

In [ ]:
imaging.Activity()

In [ ]:
imaging.MotionCorrection.heading

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

# Testing if data is loaded correctly

In [ ]:
session_key = (imaging.Fluorescence & 'subject = "jm003"').fetch('KEY')[0]
print(session_key)

In [ ]:
imaging.MotionCorrection & session_key

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
avg_im

In [ ]:
avg_im = (imaging.MotionCorrection.Summary & session_key).fetch('average_image')
plt.imshow(np.array(avg_im[0]))
plt.show()
plt.imshow(np.array(avg_im[1]))
plt.show()
plt.imshow(np.array(avg_im[2]))
plt.show()

In [ ]:
from scipy.stats import zscore

In [ ]:
imaging.Activity.Trace & session_key

In [ ]:
F = (imaging.Fluorescence.Trace & session_key).fetch('fluorescence')

In [ ]:
F_toplot = zscore(np.stack(F), 1)

In [ ]:
plt.figure(figsize=(20,20), dpi=300)
plt.imshow(F_toplot,cmap='Greys', vmin=0, vmax=3.65)